# IRIS DATASET EXPLORATION

In [ ]:
{'name':'Álvaro Riobóo de Larriva',
'start_date':'2021/03/09'}

In [ ]:
#--< main modules >--#
import numpy as np 
import pandas as pd 
import scipy as sp
import statsmodels.api as sm
import sklearn

#--< visualization >--#
import matplotlib.pyplot as plt
import seaborn as sns

#--< others >--#
from datetime import datetime
import time
from IPython.display import display 
import pickle
import gc

for i in locals().copy():
    try:
        print("%s:"%eval(i).__name__, eval(i).__version__)
    except:
        continue

In [ ]:
###----------------< START of 'iris_nb.ipynb'>---------------###

## 0. LOAD DATASET AND PARAMETERS

In [ ]:
from sklearn import datasets
iris_dict = datasets.load_iris()
species_coded_dict = {k:v for k,v in enumerate(iris_dict['target_names'])}

# Load iris dataframe.
data = pd.DataFrame(iris_dict['data'], columns=['sepal_length','sepal_width','petal_length','petal_width'])
target = pd.DataFrame(iris_dict['target'], columns=['species'])
iris = pd.DataFrame.join(data, target)

# Display info
print(iris_dict.keys())
print("\nFeature_names:", iris_dict['feature_names'])
print("\nSpecies are coded as:", species_coded_dict)
print("\nNAN values:\n%a" %iris.isna().sum())
display(iris)


## 1. EXPLORATORY DATA ANALYSIS (EDA)

In [ ]:
# Display plain information, types and main stats.
descr = iris.describe()

iris.info()
display(descr)

qlabels = ['min','25%','50%','75%','max']
qvalues = descr.loc[qlabels] # we keep quartile values from df description 

In [ ]:
# Pairplot
sns.pairplot(iris)

Del anterior gráfico de pares vemos en la diagonal principal que "sepal_length" y "sepal_width" siguen una distribución t-student (si hubiese más datos sería mas parecida a la distribución normal). En cambio "petal_length" y "petal_width" parecen seguir una distribución bimodal. Nuestro target "species" sigue distribución uniforme, y de hecho vemos el mismo número de valores en todas las categorías. Éste es un dataset muy preparado y con probabilidad se habrán tocado datos a propósito desde el antiguo estudio del que es originario. 

También vemos un razonable número de casos pertenecientes a cada categoría del target. En los gráficos no diagonales restantes, podemos ver una correlación más o menos clara entre variables, incluso diversos grupos señalados dentro de los datos. Investigaremos ahora esta correlación.

NOTA: No nos pararemos demasiado en verificar si las distribuciones mencionadas anteriormente son correctas en cada variable (diagonal principal). Pero de hacerlo, el procedimiento a seguir sería hacer un test $\chi^2$ para bondad del ajuste en cada variable.

1. Se hace un histograma de nuestras variables continuas en un cierto número de bins, normalizando al conteo total de valores ([0,1]). 
2. Se crea una variable con la distribución modelo dada la función de probabilidad de esa distribución, y de longitud el número de bines. Nótese que no es una random variable, sino una que sigue perfectamente la distribución.
3. Se realiza el $\chi^2$-test para determinar con un cierto grado de confianza $\alpha$ si las distribuciones son independientes. En este caso $H_0$: expected dist. = observed dist, por lo que el test sólo puede determinar independencia entre las distribuciones, pero no asegura que la hipótesis nula (dependencia) sea correcta. En todo caso, cuanto mayor sea el p-value, más seguros estaremos de que las distribuciones son idénticas.

In [ ]:
# BOXPLOT
fig ,ax = plt.subplots(figsize=(20,10))
iris.boxplot(by='species', ax=ax)

In [ ]:
# HISTOGRAMS
print(plt.style.available)
with plt.style.context('classic'):
    data.hist(bins=20, figsize=(10,5))

In [ ]:
# HEATMAP (upper triangle)
array = iris.iloc[:,:-1].corr()
sns.heatmap(array, mask=np.tril(array), annot=True, fmt=".2f", cmap=plt.cm.inferno, square=True)

Del heatmap puede verse de manera visual como "petal_length" y "petal_width" están más correladas.

También vemos una dependencia de 'sepal_length' con el target, y incluso vemos que 'sepal_width' está anticorrelada a los demás predictores.


In [ ]:
# SEPAL & PETAL graphics
sns.lmplot(x="sepal_width", y="sepal_length", hue="species",data=iris)
sns.lmplot(x="petal_width", y="petal_length", hue="species",data=iris)

- Gráfico sepal length/width: Las características del sépalo diferencian a la setosa, pero no a la versicolor y virginica entre sí.
- Gráfico petal length/width: La setosa se caracteriza por tener dimensiones menores tanto en longitud como en anchura del pétalo. La versicolor y virginica parecen más diferenciables que en el gŕáfico anterior, aunque hay una cierta región de convivencia.

Veremos si el test t-student y el chi-cuadrado nos pueden arrojar certidumbre sobre las poblaciones.

In [ ]:
# TEST T-STUDENT: H0-> true diff in means=0 (two-sided test)/ rejectable if p-value<0.05 at 95% confidence.
from scipy.stats import ttest_ind
print(species_coded_dict)

setosa = iris.loc[iris['species']==0]
versicolor = iris.loc[iris['species']==1]
virginica = iris.loc[iris['species']==2]

pw_ttest = ttest_ind(versicolor['petal_width'], virginica['petal_width']) 
pl_ttest = ttest_ind(versicolor['petal_length'], virginica['petal_length'])
sw_ttest = ttest_ind(versicolor['sepal_width'], virginica['sepal_width']) 
sl_ttest = ttest_ind(versicolor['sepal_length'], virginica['sepal_length'])

print('\nVersicolor vs. Virginica: Ttest')
print('petal_width--->{}\npetal_length--->{}'.format(pw_ttest, pl_ttest))
print('sepal_width--->{}\nsepal_length--->{}'.format(sw_ttest, sl_ttest))

## alternative=greater for 'sepal_width' (versicolor < virginica ?)
from scipy.stats import t

t_crit_95 = t.ppf(df=len(virginica)+len(versicolor)-2, q=0.95) #df=degrees_freedom
t_crit_99 = t.ppf(df=len(virginica)+len(versicolor)-2, q=0.99)
print('t critical at 95% conf:{}'.format(t_crit_95), 
     '\n(idem for 99%):{}'.format(t_crit_99))

El valor 'statistic' es lo que se denomina t-value. El signo de t en el test indica si la primera muestra tiende a ser mayor (+) o viceversa(-), en este caso obtenemos lo que ya sabíamos en los gráficos 'width'vs'length' anteriores. El hecho de que el t-value (de nuestros tests) sea mayor en valor absoluto que el t-value-crítico de nuestras muestras para un intervalo de confianza dado, nos indica que podemos rechazar la hipótesis nula y por tanto afirmar que las muestras poseen diferentes medias. Si tenemos que $t_{value} > t_{value-crit}$, podemos rechazar $H_0$, lo cual se dará en el caso de 'one-sided test, less-than' donde $H_0: \mu_{versicolor} >= \mu_{virginica} $, y podríamos afirmar la hipótesis alternativa $H_1: \mu_{versicolor} < \mu_{virginica}$, esto ocurre para los dos intervalos de confianza definidos (95 y 99\%) para todas las variables.

Como habíamos observado en los gráficos, los valores de 'sepal_width' y 'sepal_length' eran más difusos. Hemos comprobado con los anteriores tests, que aunque en media 'sepal_width' para las clases virginica y versicolor se parezcan, aún son separables. Aún así, es una muestra difusa de ser un peor predictor.

In [ ]:
# TEST CHI-SQUARE FOR INDEPENDENCE BETWEEN CATEGORICALS: 
'''
H0 -> sample dists are the same (table-samples)-> so that distinct categories of a variable doesn't affect the other variable 
 -> variables are independent // rejectable if p-value<0.05 at 95% confidence. '''
from scipy.stats import chisquare, chi2, chi2_contingency
from scipy.stats import distributions

# CATEGORICAL MAPPING: We map our dataset into a new one with categorical variables matching each interquartile range.
def map_quartile_ranges(x, col):
    if x<qvalues.loc['25%',col]:
        catv = 'low'
    elif x<qvalues.loc['50%',col]:
        catv = 'low-average'
    elif x<qvalues.loc['75%',col]:
        catv = 'high-average'
    else:
        catv = 'high'
    return catv

data_cat = data.copy()
for c in data.columns:
    data_cat[c] = data[c].map(lambda x: map_quartile_ranges(x, c))

def chi2contingency_by_LemmaOrCol(lemma, one_col):
    if lemma:
        features = data_cat.loc[:,data_cat.columns.str.contains(lemma)] 
        
    else:
        features = data_cat.loc[:,:] if not one_col else pd.DataFrame(data_cat.loc[:,one_col]) 
        
    iris_cat = pd.DataFrame.join(features, iris['species'].astype('object'))

    contable = iris_cat.value_counts().unstack('species').fillna(0)
    display(contable)
    res = chi2_contingency(contable)
    gc.collect()
    return res

chi2contingency_by_LemmaOrCol(lemma = None, one_col="sepal_width") 
# check lemma = 'sepal','petal','length','width', None
# check all columns separately for lemma=None

Podemos comprobar si las variables tienen dependencia con el target en una versión categórica del dataset, donde asignamos cuatro clases dependiendo del rango intercuartil en el que caigan [0->0.25,-> 0.5,->0.75,->1.00]. En todas las agrupaciones de columnas que se pueden comprobar con la función 'chi2contingency_by_LemmaOrCol', podemos ver un p-value demasiado bajo ($t<e^{-12}$), por lo cual podemos descartar la hipótesis nula que nos dice que las muestras son iguales y por tanto, nuestras variables categóricas son dependientes. Ésto es así porque hemos clasificado en categorías respecto a los rangos de cada variable, sin tener en cuenta las demás. Es decir, hemos comprobado algo tan obvio como que las tres especies de plantas son distintas entre sí atendiendo a la clasificación relativa de algunas de sus propiedades (como 'sepal_width'). Si hubiese alguna combinación de características categóricas que contribuyese significativamente a separar las especies (dependencia), se vería reflejada en este test al tener un p-value alto en torno a un intervalo de confianza aceptable.

In [ ]:
from scipy.stats import chi2_contingency

for f in data.columns:
    fig = plt.figure()
    ax = fig.add_subplot()
    ctab = pd.crosstab(iris['species'],iris[f])
    sns.heatmap(ctab, ax=ax)
    c, p, dof, expected = chi2_contingency(ctab,)
    print(f,'p_value for test is:%f'%p)

La tabla de contingencia representada en heatmaps puede tener una interpretación menos clara. Se puede decir, que representa qué valores de una variable determinada contribuyen más al valor esperado de esa variable diferenciándola por nuestro target 'species'.

In [ ]:
import statsmodels.formula.api as smf
import patsy


rhs = "C(species)"  # independent, categorical
lhs = "sepal_width" # dependent, numeric

formula = lambda lhs,rhs: lhs + "~" + rhs   # formulas are handled by "patsy" module (.dmatrices)
#C(variable, method) wrapper that var as cathegory for patsy.dmatrices || default: method='Treatment'



# OLS: cannot do "species~(any)" or "(any) + 1 ~ species". It compares same-shape arrays (ONE-WAY)
[ print("{}:\n{}\n".format(lhs, smf.ols( formula(lhs , rhs) ,data=iris).fit().summary())) for lhs in data.columns]


Un simple OLS para cada variable nos dice cómo de bien ajustaría esta variable, midiendo el $R^2$ del modelo resultante. Por el momento, vemos que "sepal_width" es el peor predictor, mientras que los mejores predictores serían la anchura y longitud del pétalo.

## 2. FINDING CANDIDATE ALGORITHMS

Para la búsqueda de algoritmos candidatos, usaremos los siguientes para el problema de clasificación:

- LogisticRegression()
- KNeighborsClassifier()
- DecisionTreeClassifier()
- RandomForestClassifier()
- SVC()
- XGBClassifier()

Podemos buscar los hiperparámetros más tarde por la clase **"GridSearchCV"**, recomendada para datasets con pocos datos y que comprueba todas las opciones disponibles.

In [ ]:
#--< scikit-learn imports >--#
from sklearn.metrics import make_scorer, roc_curve, RocCurveDisplay, auc
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.preprocessing import StandardScaler, LabelEncoder, LabelBinarizer, label_binarize
from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.linear_model import LogisticRegression, RidgeClassifier, Lasso
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier


In [ ]:
# DATA/TARGET definitions
data = iris.drop("species", axis=1)
target = iris["species"].values

labels = pd.Series([0,1,2])
n_classes = len(labels)

# SCALERS
sc = StandardScaler()
data_sc = sc.fit_transform(data)

#labenc = LabelBinarizer()
#target = labenc.fit_transform(target)

# TRAIN/TEST split
X_train, X_test, y_train, y_test = train_test_split(data_sc, target, 
                                                    test_size=0.3, random_state=1994)




In [ ]:
# We define a function to evaluate all models in a dictionary.
def train_all_models(models_dict, train_now=True):
    if train_now:
        for k,m in models_dict.items():
            start_time = time.time()
            print('MODEL:%s'%k)
            m.fit(X_train, y_train)
            print("(model_runtime= %.2f s)\n"%(time.time() - start_time))
    else: 
        print("Not training for now")
    return models_dict

def plot_predict_all_models(models_dict, kw_dict={'average':'weighted'}):
    """N=1 example:
    y_pred = label_binarize(XGBc.predict(X_test), classes=labels)
    res = roc_auc_score(y_test, y_pred, average="macro", multi_class="ovo")
    print(res)"""
    #lb = lambda arr: label_binarize(arr, classes=labels)
    for k,m in models_dict.items():
        y_pred = m.predict(X_test)
        y_pred_bin = m.predict_proba(X_test) #label_binarize(y_pred, classes=labels)
        print('MODEL :%s\n'%k,
              'AUC = %.3f\n'%roc_auc_score(y_test, y_pred_bin, 
                                           average=kw_dict['average'], 
                                           multi_class="ovo"),
              'Acc= %.3f\n'%accuracy_score(y_test, y_pred ),
              'Prec= %.3f\n'%precision_score(y_test, y_pred, 
                                             average=kw_dict['average']), 
              'Rec= %.3f\n'%recall_score(y_test, y_pred, 
                                         average=kw_dict['average']),
              'F1= %.3f\n'%f1_score(y_test,y_pred, 
                                    average=kw_dict['average'])
             )
        
        
        fig, axs = plt.subplots(nrows=1, ncols=1, figsize=(10,5))
        axs.set_title(r"ROC curve || Model: {}".format(m.__repr__()))
        axs.set_xlabel("FPR") ; axs.set_ylabel("TPR")
        
        
        for c in range(n_classes):
            fpr, tpr, thresholds = roc_curve(y_test, y_pred_bin[:,c], pos_label=c) 
            roc_auc = auc(fpr, tpr)
            axs.plot(fpr, tpr, label=f"class:{c} || AUC={roc_auc:.3f}")
            axs.plot([0,1], [0,1], color="navy", linestyle='--')
        fig.legend()



In [ ]:
## VANILLA MODELS
Lc = LogisticRegression()
KNc = KNeighborsClassifier()
DTc = DecisionTreeClassifier()
RFc = RandomForestClassifier()
SVc = SVC(probability=True)
XGBc = XGBClassifier(objective="reg:logistic",
                     eval_metric="rmse",
                     use_label_encoder=False)  #objective="multi:softmax", "reg:logistic", "multi:softprob" // probabilities=Trues ->(W!)

models_vanilla = {"Lc" : Lc,
             "KNc" : KNc,
             "DTc" : DTc,
             "RFc" : RFc,
             "SVc" : SVc,
             "XGBc" : XGBc}

# TRAIN
models_vanilla = train_all_models(models_vanilla, train_now=True)

In [ ]:
# EVALUATION: VANILLA MODELS
plot_predict_all_models(models_vanilla, kw_dict={'average':'weighted'})

## 3. EVALUATION AND IMPROVEMENTS FOR THOSE ALGORITHMS

In [ ]:
%timeit
## SEARCH FOR PARAMETERS GRIDSEARCHCV: CV MODELS || NOTE: Time + + + ...
Lc_kw = {'penalty': ['l1', 'l2', 'elasticnet', 'none'],
        'fit_intercept' : [True, False],
        'solver' : ['newton-cg', 'lbfgs', 'liblinear'],
        'class_weight' : ['balanced', None],
        'multi_class': ['multinomial', 'ovr'],}
Lc_cv = GridSearchCV(Lc, Lc_kw, cv=4, ) 

KNc_kw = {'n_neighbors' : np.arange(1,7),
          'weights' : ['uniform', 'distance'],
          'algorithm' : ['auto', 'ball_tree', 'kd_tree', 'brute'],
          'leaf_size' : np.arange(10,30),
          'p' : np.arange(1,3)
         }
KNc_cv = GridSearchCV(KNc, KNc_kw, cv=4, ) 

DTc_kw = {"criterion" : ['gini', 'entropy'],
          "splitter" : ["best", "random"], 
          "max_depth" : [2,3,4,None],
          "min_samples_leaf" : np.arange(1,9), 
          "min_samples_split": np.arange(2,5),
          "max_features" : ["sqrt", "log2", None],
          "max_leaf_nodes":[20, 30, None],
          "class_weight" : ['balanced', None],
          "ccp_alpha" : [0, 1]
          }
DTc_cv = GridSearchCV(DTc, DTc_kw, cv=4, ) 

RFc_kw = {"n_estimators" : [20,40], 
          "criterion" : ['gini','entropy'],
          "min_samples_leaf" : np.arange(1, 7, 2), 
          "min_samples_split": np.arange(2, 3),
          "max_features" : ["sqrt", "log2"], 
          "max_depth" : [3, 4, None],
          "max_features" : ["sqrt", "log2"],
          "max_leaf_nodes":[10, None],
          "class_weight" : ['balanced', None],
         }
RFc_cv = GridSearchCV(RFc, RFc_kw, cv=3, )  

SVc_kw = {"kernel" : ["linear", "poly", "rbf", "sigmoid"],
          "degree": [2, 3, 4, 5],
          "gamma" : ["scale", "auto"],
          "shrinking" : [True, False],
          "C" : [0.5, 1, 1.5, 2],
          "class_weight" : ['balanced', None],
         }
SVc_cv = GridSearchCV(SVc, SVc_kw, cv=3, ) 

XGBc_kw = {'n_estimators' : [50, 100],
    'max_depth' : [2, 3, 4],
    'learning_rate' : [0.02, 0.05, 0.1],
    'booster' : ['gbtree','gblinear','dart'],
    'base_score' : [0.25, 0.5, 0.75, 1]}

XGBc_cv = GridSearchCV(XGBc, XGBc_kw, cv=5 ) 

models_CV = {"Lc" : Lc_cv,
             "KNc" : KNc_cv,
             "DTc" : DTc_cv,
             "RFc" : RFc_cv,
             "SVc" : SVc_cv,
             "XGBc" : XGBc_cv}

# TRAIN
models_CV = train_all_models(models_CV)

In [ ]:
## EVALUATION AND PLOTTING: CV MODELS
plot_predict_all_models(models_CV, kw_dict={'average':'weighted'})

In [ ]:
%timeit
### HYPERPARAMETER TUNING: IMPROVED MODELS
Lc_imp = LogisticRegression(**models_CV['Lc'].best_params_) 
KNc_imp = KNeighborsClassifier(**models_CV['KNc'].best_params_)
DTc_imp = DecisionTreeClassifier(**models_CV['DTc'].best_params_)
RFc_imp = RandomForestClassifier(**models_CV['RFc'].best_params_)
SVc_imp = SVC(**models_CV['SVc'].best_params_, probability=True)
XGBc_imp =XGBClassifier(**models_CV['XGBc'].best_params_,
                        probability=True,
                        use_label_encoder=False,
                        objective="reg:logistic",
                        eval_metric="rmse")


models_improved = {"Lc" : Lc_imp,
             "KNc" : KNc_imp,
             "DTc" : DTc_imp,
             "RFc" : RFc_imp,
             "SVc" : SVc_imp,
             "XGBc" : XGBc_imp}

# TRAIN
models_improved = train_all_models(models_improved, train_now=True)

In [ ]:
## EVALUATION AND PLOTTING: IMPROVED MODELS
plot_predict_all_models(models_improved, kw_dict={'average':'weighted'})

In [ ]:
%timeit
### BAGGING ENSEMBLE: BAG MODELS
Lc_bag = BaggingClassifier(LogisticRegression(**models_CV['Lc'].best_params_)) 
KNc_bag = BaggingClassifier(KNeighborsClassifier(**models_CV['KNc'].best_params_))
DTc_bag = BaggingClassifier(DecisionTreeClassifier(**models_CV['DTc'].best_params_))
RFc_bag = BaggingClassifier(RandomForestClassifier(**models_CV['RFc'].best_params_))
SVc_bag = BaggingClassifier(SVC(**models_CV['SVc'].best_params_, probability=True))
XGBc_bag = BaggingClassifier(XGBClassifier(**models_CV['XGBc'].best_params_,
                                            probability=True,
                                            use_label_encoder=False,
                                            objective="reg:logistic",
                                            eval_metric="rmse"))


models_bag = {"Lc" : Lc_bag,
             "KNc" : KNc_bag,
             "DTc" : DTc_bag,
             "RFc" : RFc_bag,
             "SVc" : SVc_bag,
             "XGBc" : XGBc_bag}

# TRAIN
models_bag = train_all_models(models_bag, train_now=True)

In [ ]:
## EVALUATION AND PLOTTING: BAGGING MODELS
plot_predict_all_models(models_bag)

In [ ]:
%timeit
# MAIN RESULTS INTO DATAFRAMES
algorithms = ['Logistic Regression',
            'K-Nearest Neighbour Classifier',
            'Decision Tree Classifier', 
            'Random Forest Classifier',
            'Support Vector Machine Classifier',
            'XGBoost Classifier']

vanilla_results = pd.DataFrame( {"Algorithm": algorithms,
                          "Train_score":[m.score(X_train,y_train) for m in models_vanilla.values()],
                          "Test_score":[m.score(X_test,y_test) for m in models_vanilla.values()]
                                }).sort_values('Test_score', ascending=False)

improved_results = pd.DataFrame({"Algorithm": algorithms,
                          "Train_score":[m.score(X_train,y_train) for m in models_improved.values()],
                          "Test_score":[m.score(X_test,y_test) for m in models_improved.values()]
                                }).sort_values('Test_score', ascending=False)

bag_results = pd.DataFrame({"Algorithm": algorithms,
                          "Train_score":[m.score(X_train,y_train) for m in models_bag.values()],
                          "Test_score":[m.score(X_test,y_test) for m in models_bag.values()]
                               }).sort_values('Test_score', ascending=False)

display("VANILLA:", vanilla_results)
display("IMPROVED:", improved_results)
display("BAGGING:", bag_results)

In [ ]:
# EXPORT CLEANED AND IMPUTED DATASET TO CSV (for production)
iris.to_csv("iris.csv", header=True, index=False)

# EXPORTS TO CSV
vanilla_results.to_csv("iris_vanilla_results.csv")
improved_results.to_csv("iris_improved_results.csv")
bag_results.to_csv("iris_bagging_results.csv")

# EXPORT TO PICKLE
import pickle

model_filename = 'iris_svm_bag.pkl'
with open(model_filename, "wb") as f:
    pickle.dump(models_bag['SVc'], f)
with open("iris_lc_imp.pkl", "wb") as f:
    pickle.dump(models_improved['Lc'], f)
    


In [ ]:
# FEATURE IMPORTANCES
features = pd.Series(models_bag['RFc'].base_estimator_.fit(X_train,y_train).feature_importances_)
features.index = features.index.map({i:c for i,c in enumerate(data.columns)})
display(features)

## <ins>*CONCLUSIONES*:</ins>

**Variables predictoras o features:** 'sepal_width','sepal_length','petal_width','petal_length' \
**Variable objetivo o target:** 'species'

- 1. ANÁLISIS EXPLORATORIO DE DATOS:

    - Hay correlación (bastante mayor en las características del pétalo) entre las variables predictoras y la objetivo, salvo en el caso de 'sepal_width',en la que hay una mínima correlación negativa.
    
    - La distribución bimodal en algunos de nuestros histogramas y un scatterplot más detallado en las características del pétalo diferenciando por especies, nos llevan a la conclusión de que la especie setosa posee unas propiedades del pétalo suficientemente características para ser diferenciadas del resto de especies. Se podría quitar el subgrupo setosa de nuestros datos y seguir investigando de manera visual y analítica la separación entre versicolor y virginica. 
    
    - Las variables 'petal_length' y 'petal_width' (sobre todo la segunda) pueden parecer a simple vista que comparten rangos entre las especies virginica y versicolor, pero un análisis más detallado con el test T-student indica que son separables estadísticamente con una confianza significativa (95%). También este test nos dice que ambas características del pétalo son objetivamente mayores en el caso de la virginica.
    
    - Si convertimos a variables categóricas en función de nuestros rangos intercuartiles, no hay una combinación de características que sea dependiente y por tanto predictora de nuestra variable target. Ésto hemos podido comprobarlo con nuestro test $\chi^2$ de independencia.
        
- 2. BUSCANDO ALGORITMOS CANDIDATOS:

Definimos nuestro target 'species', a continuación hemos dividido en train/test con una fracción del 70% y 30%, respectivamente.

Hemos realizado un StandardScaler() que nos redistribuye la muestra como $\sim N(0,1)$, ésto es suficiente y recomendable puesto que no tenemos demasiados valores extremos en nuestros datos y nuestras variables predictoras poseen una distribución normal.

Hemos usado los siguientes y famosos algoritmos de sklearn para problemas de clasificación:

    - LinearClassifier(): Modelo más simple, ajusta los coeficientes por medio de minimizar diferencias cuadráticas.
    - KNeighborsClassifier(): Modelo sencillo que, una vez se establece k (el número de predictores máximo), nos dará la importancia de predictores en torno a la variable de objetivo de la clasificación.
    - DecisionTreeClassifier(): Modelo sencillo que penaliza la distancia de nuestros datos en relación a valores de prueba , construye un árbol de decisión y permite realizar clasificación. Permite ver la importancia de predictores.
    - RandomForestClassifier(): Modelo mejorado basado en árboles de decisión que permite promediar una muestra significativa de ellos, sus resultados, y ver la importancia de predictores.
    - SVC(): Modelo de máquinas de soporte vectorial ('Support Vector Machines'), que busca hacer la mejor división de nuestros datos mediante hiperplanos que los separen.
    - XGBClassifier(): Modelo complejo y óptimo ('eXtreme Gradient Boosting') basado en árboles de decisión, optimización del descenso del gradiente y una randomizazión de parámetros optimizada, entre otros.
    
Dado que nuestro dataset es relativamente pequeño, hemos usado GridSearchCV como algoritmo de búsqueda de hiperparámetros. Éste es mas lento que otros (i.e. RandomizedSearchCV), pero ésto se traduce en una mejora general de los hiperparámetros buscados, dado que prueba todas las combinaciones en los datos.

**NOTA: Hemos evaluado la área debajo de la curva ROC (AUC), la precisión (accuracy) del modelo y otras métricas principales de la tabla de contingencia, también hemos dibujado la curva ROC de los modelos para cada posible clase. En las métricas, hemos optado por hacer una media pesada ("weighted") entre los 3 pares de evaluación todos-contra-todos.**

- 3. EVALUACIÓN Y MEJORAS PARA ÉSTOS ALGORITMOS:

Una vez buscados los mejores parámetros para nuestros modelos, hemos evaluado nuestros modelos con éstos, lo que hemos llamado modelos "improved".

Luego, hemos usado un método de ensamblado ('BaggingClassifier') para mejorar nuestros modelos mediante el ensamblaje de clasificadores con los mejores parámetros encontrados, lo que hemos llamado modelos "bagging".

Hemos exportado nuestros modelos a DataFrames y hemos guardado los resultados principales, también, hemos guardado el dataset en formato .csv y el mejor modelo en formato 'pickle' para la fase de producción.

En nuestros algoritmos principales evaluados en test:

Para modelos "vanilla" (default): Gana *Logistic Regression*, con un score de 0.978 en test. 

Para modelos "improved" (con hiperparámetros mejorados), nos dan como favoritos:

1. Logistic Regression 	0.980952 	0.977778 \
2. K-Nearest Neighbour Classifier 	0.971429 	0.933333 \
3. Support Vector Machine Classifier 	0.961905 	0.933333 \

Para modelos "bagging" (mejorados con ensamblamiento):

1. Support Vector Machine Classifier 	0.971429 	0.977778 \
2. Logistic Regression 	0.990476 	0.955556 \
3. Random Forest Classifier 	0.961905 	0.955556 \

Por tanto, guardaremos el modelo SVM_bagging y el Lc_improved en formato 'pickle' para la fase de producción. También guardamos el dataset original.

Según nuestro mejor modelo que permite estimar la importancia de los predictores en la variable objetivo (RandomForestClassifier_bagging), irían por orden:

**'petal_width' > 'petal_length' > 'sepal_length' > 'sepal_width'**


In [ ]:
###----------------< END of 'iris_nb.ipynb'>---------------###